In [21]:
import cobra
from cobra.io import save_json_model, read_sbml_model, write_sbml_model
import os, re
import pandas as pd

print(os.getcwd())

ext_dir = '/../../../data/external'
mass_balance_dir = '/../4.mass_balance'
man_dir = '/../3.manual_check'

# Import model
# model = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM.xml")
# model_ori = read_sbml_model(f"{os.getcwd()}/{man_dir}/Salb-GEM-Man-gapfill.xml")
model = read_sbml_model(f"{os.getcwd()}/{mass_balance_dir}/Salb-GEM-Biosustain.xml")
# model_ori = read_sbml_model(f"{os.getcwd()}/{ext_dir}/Salb-GEM-Updated.xml")

/Users/te/Library/CloudStorage/OneDrive-Personal/Biosustain/strep/Salb_GEM_Biosustain/notebooks/gapfilling/4.mass_balance


'' is not a valid SBML 'SId'.


# Get all gapfilled reactions

In [2]:
diff_r = []
for r in model.reactions:
    if model_ori.reactions.has_id(r.id) == False:
        diff_r.append(r)
    else:
        if model_ori.reactions.get_by_id(r.id).bounds != r.bounds:
            diff_r.append(r)
        elif model_ori.reactions.get_by_id(r.id).build_reaction_string() != r.build_reaction_string():
            diff_r.append(r)

reaction_data = [{
    'Reaction ID': reaction.id,
    'Name': reaction.name,
    'Reaction String': reaction.reaction,
    'Reference': reaction.annotation
} for reaction in diff_r]

reaction_df = pd.DataFrame(reaction_data)
reaction_df.to_csv('all_gapfilled_reactions.csv')
print(reaction_df)

    Reaction ID                                         Name  \
0      3OCHOCDH   3-oxocholest-4-en-26-oyl-CoA dehydrogenase   
1      3OXCOADH      3-oxochol-4-en-24-oyl-CoA dehydrogenase   
2       4SBZADH            4-sulfobenzaldehyde dehydrogenase   
3      AHMMPS_3  phosphomethylpyrimidine synthase ThiC: thiC   
4         AHOPS                     adenosyl hopane synthase   
..          ...                                          ...   
154      AHSERL          O-acetyl-L-homoserine acetate-lyase   
155        ARGJ                 amino acid acetyltransferase   
156       METGL                       methionine gamma-lyase   
157     DHPDH_1              Dihydropyrimidine dehydrogenase   
158     DHPDH_2              Dihydropyrimidine dehydrogenase   

                                       Reaction String  \
0    3oce26ocoa_c + acceptor_c --> 3ocd26ocoa_c + d...   
1    3oc4en24ocoa_c + acceptor_c --> 3ocdncoa_c + d...   
2    4sulbza_c + h2o_c + nad_c <=> 4sulfbenz_c + h_...   

In [3]:
# Find out what genes could be wrong by checking growth.

possibly_wrong = []
for r in diff_r:
    with model_ori:
        model_ori.add_reactions([r])
        model_ori.objective = 'growth'
        solution = model_ori.optimize().objective_value
        print(model_ori.optimize().fluxes[r.id])
    print(r, solution)
    if solution > 0.06:
        possibly_wrong.append(r)

possibly_wrong

Ignoring reaction '3OCHOCDH' since it already exists.
Ignoring reaction '3OXCOADH' since it already exists.
Ignoring reaction '4SBZADH' since it already exists.
Ignoring reaction 'AHOPS' since it already exists.
Ignoring reaction 'AIRCr' since it already exists.


0.0
3OCHOCDH: 3oce26ocoa_c + acceptor_c --> 3ocd26ocoa_c + donor_c + h_c 0.05217336185319603
0.0
3OXCOADH: 3oc4en24ocoa_c + acceptor_c --> 3ocdncoa_c + donor_c + 5.0 h_c 0.05217336185319603
0.0
4SBZADH: 4sulbza_c + h2o_c + nad_c <=> 4sulfbenz_c + h_c + nadh_c 0.05217336185319603
0.0
AHMMPS_3: air_c + amet_c --> 4ampm_c + co_c + dad_5_c + for_c + 3.0 h_c + met__L_c 0.05217336185319648
0.0
AHOPS: amet_c + donor_c + hop22en_c --> acceptor_c + ahop_c + met__L_c 0.05217336185319648


Ignoring reaction 'AMBACS' since it already exists.
Ignoring reaction 'ASR' since it already exists.
Ignoring reaction 'CMCMSAD' since it already exists.
Ignoring reaction 'CYANST' since it already exists.
Ignoring reaction 'CYO1ab' since it already exists.


0.016583366852192402
AIRCr: air_c + co2_c <=> 5aizc_c + 2.0 h_c 0.05217336185319648
0.0
AMBACS: formhop_c + glu__L_c --> abacptri_c + akg_c + h_c 0.05217336185319648
0.0
ASR: aso4_c + 2.0 msh_c --> aso3_c + h2o_c + h_c + mssg_c 0.05217336185319648
0.0
CMCMSAD: 5cmhmsa_c + h2o_c + h_c + nad_c --> 5cmhm_c + nadh_c 0.05217336185319648
0.0
CYANST: cyan_c + tsul_c --> h_c + so3_c + tcynt_c 0.05217336185319648


Ignoring reaction 'CYO2b' since it already exists.
Ignoring reaction 'CYSBLY' since it already exists.
Ignoring reaction 'DABAACT' since it already exists.
Ignoring reaction 'DABAAT' since it already exists.
Ignoring reaction 'DDALLO' since it already exists.


...

Ignoring reaction 'r_0012' since it already exists.
Ignoring reaction 'r_0016' since it already exists.
Ignoring reaction 'r_0017' since it already exists.
Ignoring reaction 'r_0018' since it already exists.


0.0
r_0011: amet_c + m_0008 <=> ahcys_c + h_c + m_0009 0.05217336185319648
0.0
r_0012: accoa_c + m_0009 + nad_c + pyr_c <=> co2_c + coa_c + h_c + m_0010 + nadh_c 0.05217336185319648
0.0
r_0016: accoa_c + m_0012 <=> coa_c + m_0016 0.05217336185319648
0.0
r_0017: h_c + m_0016 + nadh_c <=> m_0017 + nad_c 0.05217336185319648
0.0


Ignoring reaction 'r_0019' since it already exists.
Ignoring reaction 'metylmalonyl_synthesis' since it already exists.
Ignoring reaction 'ethylmalonyl_synthesis' since it already exists.


r_0018: 2mbcoa_c + m_0017 <=> coa_c + m_0014 0.05217336185319648
0.0
r_0019: ibcoa_c + m_0017 <=> coa_c + m_0015 0.05217336185319648
0.0
metylmalonyl_synthesis: atp_c + 4.0 h_c + hco3_c + ppcoa_c --> adp_c + mmcoa_c + pi_c 0.05217336185319648
0.0
ethylmalonyl_synthesis: b2coa_c + co2_c + 5.0 h_c + nadph_c --> ethmalcoa_c + nadp_c 0.05217336185319648
0.0
GLYC3tr: glyc3p_e --> glyc3p_c 0.05217336185319648
0.0
PPYRDC: h_c + phpyr_c --> co2_c + pacald_c 0.0521733618531963
0.0
APTNAT: 5aptn_c + akg_c <=> glu__L_c + oxptn_c 0.0521733618531963
0.0
DAPAT: 15dap_c + akg_c --> glu__L_c + h2o_c + h_c + pprdn_c 0.05217336185319666
0.0
NP1_2: nicrns_c + pi_c --> h_c + nac_c + r1p_c 0.05217336185319712
0.0
HOMt4: hom__L_e + na1_e --> hom__L_c + na1_c 0.05217336185319712
0.0
FBA4: xu1p__D_c <=> dhap_c + gcald_c + 2.0 h_c 0.05217336185319712
0.0
EX_citr__L_e: citr__L_e -->  0.05217336185319666
0.0
ACGApts: acgam_e + pep_c --> acgam6p_c + pyr_c 0.05217336185319666
0.0
OCBT2i: citr__L_c + h_c + pi_c -->

[]

In [4]:
addition = list(set(diff_r) - set(possibly_wrong))

# Manual Check Gapfilled Reactions in all_gapfilled_reactions.csv

In [5]:
# Update reaction names
for r in addition:
    if r.name == "":
        print(r.id)


3HPCOAHYD
CBMD
RE0691C
SELGTHR


In [14]:
model.reactions.get_by_id('RE0691C').annotation

{'sbo': 'SBO:0000176', 'ec-code': '2.6.1.82'}

In [113]:
model.metabolites.dnac_c.formula = 'R'
model.metabolites.dnac_c.charge = -1

model.metabolites.dna5mtc_c.formula = 'CH3R'
model.metabolites.asntrna_c.formula = 'C19H33N2O18P2'
model.metabolites.asntrna_c.charge = 2
model.metabolites.asptrna_asn_c.formula = 'C19H30NO19P2'

model.metabolites.get_by_id('2ahethmpp_c').formula = 'C14H23N4O8P2S'

model.metabolites.trnagln_c.formula = 'C15H20N5O10PR'
model.metabolites.glntrna_c.formula = 'C20H29N7O12PR'
model.metabolites.glntrna_c.charge = 1
model.metabolites.glutrna_gln_c.formula = 'C20H26N6O13PR'
model.metabolites.glutrna_gln_c.charge = -1


model.metabolites.sertrna_sec_c.formula = 'C18H28NO18P2'
model.metabolites.sertrna_sec_c.charge = -2
model.metabolites.sectrna_c.formula = 'C18H30NO17P2Se'

In [114]:
for r in model.reactions:
    balance = r.check_mass_balance()
    if balance != {} and not (r.id.startswith('EX_') or r.id.startswith('DM_') or "PSEUDO" in r.id):
        print(r.id,r.name, balance)

BIOMASS_SALB S. albus biomass objective function - with 173 GAM estimate {'R': -7.0}
r_0004 oxidoreductase {'charge': -1.0, 'H': 3.0}
r_0005 hydroxylase {'charge': 3.0}
r_0007 D-allose C-glycosyltransferase {'charge': -1.0, 'H': -3.0}
r_0012 pyruvate:NAD+ 2-oxidoreductase (CoA-acetylating) {'charge': -1.0, 'C': 11.0, 'H': 22.0, 'O': 4.0}
r_0013 NDP-L-paulomycose C-glycosyltransferase {'charge': 3.0, 'C': -25.0, 'H': -38.0, 'N': -2.0, 'O': -17.0, 'P': -2.0, 'S': 1.0}
r_0015 paulyl-CoA biosynthesis {'charge': -1.0, 'C': 3.0, 'H': 1.0, 'N': 1.0, 'O': 1.0}


In [23]:
model.reactions.get_by_id('3HPCOAHYD').name = '3-hydroxyisobutyryl-CoA hydrolase'
model.reactions.CBMD.name = 'Carbamate deaminase'
model.reactions.SELGTHR.name = 'Selenate glutathione reductase'
model.reactions.get_by_id('RE0691C').name = 'putrescine—2-oxoglutarate transaminase'

In [120]:
from cobra.io import write_sbml_model, save_json_model, save_matlab_model, save_yaml_model
model_dir = '../../../model/'
write_sbml_model(model, f'{model_dir}/Salb-GEM-Biosustain.xml')
save_json_model(model, f'{model_dir}/Salb-GEM-Biosustain.json')
save_matlab_model(model, f'{model_dir}/Salb-GEM-Biosustain.mat')
save_yaml_model(model, f'{model_dir}/Salb-GEM-Biosustain.yaml')